# Read out data from files

In [4]:
import sqlite3
import requests
import os
from  database_handler import Database

# 1. Connect to your local database. Replace with your DB file or connection details.
# If you're using a database other than SQLite, use the appropriate client library and connection.
conn = sqlite3.connect("../data/spotify_scrape_TEST.db")
cursor = conn.cursor()

# 2. Query to get all unique track IDs alongside their preview_url.
#    We assume the track table is named 'track' and has columns 'track_id' and 'preview_url'.
#    The recently_played table references the track_id.
query = """
    SELECT DISTINCT track_id FROM recently_played
"""
cursor.execute(query)

results = cursor.fetchall()



In [6]:
# Get preview per track

from spotify_preview import get_spotify_preview_url

track_preview_dict = {}

for entry in results:
    entry = entry[0]
    preview_url = get_spotify_preview_url(entry)
    if preview_url:
        track_preview_dict[entry] = preview_url
    else:
        print(f"Preview URL not found for track ID: {entry}")

Preview URL not found for track ID: 7Lfwb4IjZiNYGuqtP65kIc
Preview URL not found for track ID: 27N2bd3ait2QOnprc4seCK
Failed to fetch Spotify preview URL: 504 Server Error: Gateway Timeout for url: https://open.spotify.com/embed/track/3Y3iC7f2ypojLMgr7yn9ZC
Preview URL not found for track ID: 3Y3iC7f2ypojLMgr7yn9ZC
Failed to fetch Spotify preview URL: 504 Server Error: Gateway Timeout for url: https://open.spotify.com/embed/track/60sKhq6DPoP4eSQGAqxE6M
Preview URL not found for track ID: 60sKhq6DPoP4eSQGAqxE6M


In [9]:


# 3. Prepare a local folder to hold audio previews
download_folder = "audio_previews"
os.makedirs(download_folder, exist_ok=True)

# 4. Iterate through the track list
for track_id, preview_url in track_preview_dict.items():

    # Form the output filename for the preview
    output_filename = os.path.join(download_folder, f"{track_id}.mp3")
    
    # Skip if the file already exists
    if os.path.exists(output_filename):
        print(f"Preview for track {track_id} already exists. Skipping...")
        continue

    # Download the preview
    try:
        response = requests.get(preview_url, timeout=30)  # you can adjust timeout
        if response.status_code == 200:
            with open(output_filename, "wb") as f:
                f.write(response.content)
            print(f"Downloaded preview for track {track_id}")
        else:
            print(f"Failed to download track {track_id}: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error downloading track {track_id}: {e}")

# Always good practice to close your DB connection
conn.close()

Downloaded preview for track 3MtP9T0WF4xbIo8vqHYeXM
Downloaded preview for track 45Ucre28FuNGYxHG7ZlAQ4
Downloaded preview for track 7moJWeYfMTr7iWgSHvwF7Y
Downloaded preview for track 1GLA53wyoybkJIQPLJR1Ml
Downloaded preview for track 4s6LhHAV5SEsOV0lC2tjvJ
Downloaded preview for track 45YtTdmH2PNKEnxN3uTZQy
Downloaded preview for track 5RLzsVW6UNiV2YrOlKwzNN
Downloaded preview for track 0AQquaENerGps8BQmbPw14
Downloaded preview for track 3ZpTyU4mF3PR9jLpFA3AAC
Downloaded preview for track 0mmveflIg5uKskiCOdRV3e
Downloaded preview for track 4nlH0pGxVuDKgJgY9HEfjS
Downloaded preview for track 2T0VhmgStXM5N3t92MqcZP
Downloaded preview for track 3rNIZAmaabVfehunonNQQB
Downloaded preview for track 2QxPR3dXMtWXWVpu5AH6BD
Downloaded preview for track 74VR3AkGPhbYXnxcOYa16x
Downloaded preview for track 1WN4uNclrDuczTO3bCr8s1
Downloaded preview for track 66RbVtQkNBFb7BcebSkeAT
Downloaded preview for track 4G9f3VGp9wjE49JUfg3XtO
Downloaded preview for track 1C0kmtVhqjfm1U9mXpvyNe
Downloaded p